In [1]:
import pandas as pd

file_path = '../08. Evaluasi Pola (Pattern Evaluation)/Dataset/kursus.csv'

data = pd.read_csv(file_path)

data

,conversation_id_str,created_at,favorite_count,full_text,id_str,image_url,in_reply_to_screen_name,lang,location,quote_count,reply_count,retweet_count,tweet_url,user_id_str,username
0,1807201203832644018,Sat Jun 29 23:55:17 +0000 2024,0,@infomalang selamat pagi min minta tolong info...,1807201203832644018,NaN,infomalang,in,NaN,0,2,0,https://x.com/AzahraDean19363/status/180720120...,1710774704037089280,AzahraDean19363
1,1807040145256034783,Sat Jun 29 23:37:54 +0000 2024,0,@danu_kata @kaveits First impression utk yg ba...,1807196829161042011,NaN,danu_kata,in,Timbuktu,0,1,0,https://x.com/apiekluvdan/status/1807196829161...,1473134795341266950,apiekluvdan
2,1807192797939953929,Sat Jun 29 23:21:53 +0000 2024,1,Kursus Pendidikan Agama https://t.co/0YTwfKFUf0,1807192797939953929,https://pbs.twimg.com/media/GRRwRbIaEAEZiVm.jpg,NaN,in,"Kuningan, Indonesia",0,0,0,https://x.com/ataulalaa/status/180719279793995...,742419990,ataulalaa
3,1806952429512233041,Sat Jun 29 22:56:05 +0000 2024,1,@imminenotyours_ @innovacommunity Biasanya bel...,1807186304561758344,NaN,imminenotyours_,in,"Colorado, USA",0,0,0,https://x.com/dabudabu6969/status/180718630456...,711377077916794880,dabudabu6969
4,1806952429512233041,Sat Jun 29 22:54:41 +0000 2024,0,@innovacommunity Paling bener ikut kursus dah,1807185953393721541,NaN,innovacommunity,in,"Colorado, USA",0,0,0,https://x.com/dabudabu6969/status/180718595339...,711377077916794880,dabudabu6969
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1235,1804304762684477570,Sat Jun 22 00:05:52 +0000 2024,0,Assalamualaikum &amp; Good morning Have a bles...,1804304762684477570,https://pbs.twimg.com/media/GQotnpLakAUHJm_.jpg,NaN,in,"Muar, Johor",0,0,0,https://x.com/APemakanan/status/18043047626844...,1278486349599748097,APemakanan
1236,1804299666043080977,Sat Jun 22 00:01:15 +0000 2024,0,@Piyu_chann tempat kursus mah gak kenal tutup ...,1804303602141532583,NaN,Piyu_chann,in,Jakarta Capital Region,0,1,0,https://x.com/kesurupaaan/status/1804303602141...,1577180584031371265,kesurupaaan
1237,1804116632228499520,Fri Jun 21 23:55:55 +0000 2024,0,@bturn2c aku pernah tapi kurang ngerti paket c...,1804302258194256144,NaN,bibequwu,in,NaN,0,0,0,https://x.com/ggelatou/status/1804302258194256144,1513078233574834176,ggelatou
1238,1804148890708681061,Fri Jun 21 23:50:22 +0000 2024,1,@tanyarlfes Daripada joki mending buka kursus ...,1804300861398749310,NaN,tanyarlfes,in,NaN,0,0,0,https://x.com/kecoabunting/status/180430086139...,611902187,kecoabunting


In [2]:
# Import libraries yang diperlukan
import string
import pandas as pd
import numpy as np
import re
import nltk
import pickle
from nltk.tokenize import word_tokenize
from mpstemmer import MPStemmer
from nltk.corpus import stopwords
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Unduh stopwords dari NLTK
nltk.download('stopwords')
stemmer = MPStemmer()

# Memuat model analisis emosi yang sudah dilatih
model = load_model('../07. Pemodelan Data (Data Mining)/model/model-bilstm.h5')

class Emotion:
    @staticmethod
    def classify_emotion(data):

        def lower_case(text):
            return text.lower()

        def remove_tweet_special(text):
            text = text.replace('\\t', " ").replace('\\n', " ").replace('\\u', " ").replace('\\', "")
            text = text.encode('ascii', 'replace').decode('ascii')
            text = ' '.join(re.sub(r"([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)", " ", text).split())
            return text.replace("http://", " ").replace("https://", " ")

        def remove_number(text):
            return re.sub(r"\d+", "", text)

        def remove_punctuation(text):
            return text.translate(str.maketrans("", "", string.punctuation))

        def remove_whitespace_LT(text):
            return text.strip()

        def remove_whitespace_multiple(text):
            return re.sub(r'\s+', ' ', text)

        def remove_singl_char(text):
            return re.sub(r"\b[a-zA-Z]\b", "", text)

        def remove_repeated_char(text):
            return re.sub(r'(.)\1+', r'\1', text)

        def word_tokenize_wrapper(text):
            return word_tokenize(text)

        normalizad_word = pd.read_csv("./utils/kamus-alay.csv")
        normalizad_word_dict = {}

        for index, row in normalizad_word.iterrows():
            if row[0] not in normalizad_word_dict:
                normalizad_word_dict[row[0]] = row[1]

        def normalized_term(document):
            return [normalizad_word_dict[term] if term in normalizad_word_dict else term for term in document]

        def stem_wrapper(term):
            return [stemmer.stem(word) for word in term]

        stop_words = stopwords.words('indonesian')
        stop_words = [word for word in stop_words if word not in ['tidak', 'baik', 'jelek', 'jangan', 'belum', 'bukan', "enggak", "engga", "bener", "benar"]]
        stop_words.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo', 'kalo', 'amp', 'biar', 'bikin', 'bilang', 
                            'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 
                            'jd', 'jgn', 'sdh', 'aja', 'n', 't', 'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                            '&amp', 'yah'])

        txt_stopword = pd.read_csv("utils/stopwords.txt", names=["stopwords"], header=None)
        stop_words.extend(txt_stopword["stopwords"][0].split(' '))
        stop_words = set(stop_words)

        def stopwords_removal(words):
            return [word for word in words if word not in stop_words]
        
        def replace_nan_with_none(data):
            return data.applymap(lambda x: None if pd.isna(x) else x)

        df = pd.DataFrame(data)

        if 'predicted_label' not in df.columns:
            df['predicted_label'] = np.nan
            df['probability_emotion'] = np.nan

        to_process_df = df[df['predicted_label'].isna()]

        if not to_process_df.empty:
            to_process_df['processed_text'] = to_process_df['full_text'].apply(lower_case)
            to_process_df['processed_text'] = to_process_df['processed_text'].apply(remove_tweet_special)
            to_process_df['processed_text'] = to_process_df['processed_text'].apply(remove_number)
            to_process_df['processed_text'] = to_process_df['processed_text'].apply(remove_punctuation)
            to_process_df['processed_text'] = to_process_df['processed_text'].apply(remove_whitespace_LT)
            to_process_df['processed_text'] = to_process_df['processed_text'].apply(remove_whitespace_multiple)
            to_process_df['processed_text'] = to_process_df['processed_text'].apply(remove_singl_char)
            to_process_df['processed_text'] = to_process_df['processed_text'].apply(remove_repeated_char)
            to_process_df['processed_text'] = to_process_df['processed_text'].apply(word_tokenize_wrapper)
            to_process_df['processed_text'] = to_process_df['processed_text'].apply(normalized_term)
            to_process_df['processed_text'] = to_process_df['processed_text'].apply(stem_wrapper)
            to_process_df['processed_text'] = to_process_df['processed_text'].apply(stopwords_removal)
            to_process_df['processed_text'] = to_process_df['processed_text'].apply(' '.join)

            print("Text preprocessing done!")

            with open('../07. Pemodelan Data (Data Mining)/tokenizer-emotion.pickle', 'rb') as handle:
                tokenizer = pickle.load(handle)

            sequences = tokenizer.texts_to_sequences(to_process_df['processed_text'])
            padded_sequences = pad_sequences(sequences, maxlen=50, truncating='post', padding='post')

            predictions = model.predict(padded_sequences)
            
            emotion_labels = ['Neutral', 'Anger', 'Joy', 'Love', 'Sad', 'Fear']
            predicted_labels = []
            predicted_probabilities = []

            for pred in predictions:
                max_idx = np.argmax(pred)
                predicted_labels.append(emotion_labels[max_idx])
                predicted_probabilities.append({emotion_labels[i]: pred[i] for i in range(len(emotion_labels))})

            print("Prediction done!")

            to_process_df['predicted_label'] = predicted_labels
            to_process_df['probability_emotion'] = [predicted_probabilities[i][predicted_labels[i]] for i in range(len(predicted_labels))]

            df.update(to_process_df)

        df = replace_nan_with_none(df)

        return df
    
    @staticmethod
    def calculate_emotion_percentages(data):
        total = len(data)
        emotion_counts = {'Neutral': 0, 'Anger': 1, 'Joy': 2, 'Love': 3, 'Sad': 4, 'Fear': 5}

        for item in data:
            emotion_counts[item['predicted_label']] += 1
        
        percentages = {emotion: (count / total) * 100 for emotion, count in emotion_counts.items()}
        return percentages


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\TEMP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# Mengklasifikasikan emosi
hasil = Emotion.classify_emotion(data)

# Menyimpan hasil hanya full_text, prediksi dan probabilitas 
hasil = hasil[['username','full_text','topic','topic_probability','predicted_label', 'probability_emotion']]
hasil.to_csv('hasil/hasil_prediksi-diet(bilstm).csv', index=False)

C:\Users\TEMP\AppData\Local\Temp\ipykernel_5644\3662870842.py:59: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if row[0] not in normalizad_word_dict:
C:\Users\TEMP\AppData\Local\Temp\ipykernel_5644\3662870842.py:60: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  normalizad_word_dict[row[0]] = row[1]


Text preprocessing done!
39/39 ━━━━━━━━━━━━━━━━━━━━ 3s 33ms/step
Prediction done!


C:\Users\TEMP\AppData\Local\Temp\ipykernel_5644\3662870842.py:132: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['Neutral' 'Sad' 'Anger' ... 'Anger' 'Neutral' 'Joy']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.update(to_process_df)
C:\Users\TEMP\AppData\Local\Temp\ipykernel_5644\3662870842.py:83: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return data.applymap(lambda x: None if pd.isna(x) else x)


KeyError: "['topic', 'topic_probability'] not in index"